### Data Aggregation

In [502]:
dd = pd.read_excel("normalised_database.xlsx")

In [503]:
therm = dd[dd.normalised_model == "ThermCond"].copy()

In [504]:
therm.type.value_counts()

total         2255
lattice       1316
electronic      75
Name: type, dtype: int64

In [505]:
# known example
therm[therm.first_name.str.contains("^Pb0.98Na0.02S$")]

,Unnamed: 0,merged_name,compound_name,first_name,normalised_model,model,type,normalised_temp_values,normalised_temp_avg,normalised_temp_units,temp_value,normalised_values,normalised_avg,normalised_units,editing,pressure,exrept,doi,error,process,direction_of_measurement,labels,parser,specifier,title,publisher,yop,oa
1656,1677,NaN,['Pb0.98Na0.02S'],Pb0.98Na0.02S,ThermCond,ThermCond,lattice,[310.0],310.0,Kelvin^(1.0),[310.0],[1.7],1.70,Kelvin^(-1.0) Meter^(-1.0) Watt^(1.0),NaN,NaN,( which results in high κele of ∼1.7 W m−1 K−1...,10.1016-j.jallcom.2019.153453.xml,NaN,NaN,NaN,NaN,comma-level,κele,An approach of enhancing thermoelectric perfor...,Elsevier,2020,no
1657,1678,NaN,['Pb0.98Na0.02S'],Pb0.98Na0.02S,ThermCond,ThermCond,lattice,[295],295.0,Kelvin^(1.0),NaN,[4.0],4.00,Kelvin^(-1.0) Meter^(-1.0) Watt^(1.0),NaN,NaN,({ at room temperature }: It is found that κt...,10.1016-j.jallcom.2019.153453.xml,NaN,NaN,NaN,NaN,comma-level,κtot,An approach of enhancing thermoelectric perfor...,Elsevier,2020,no
1660,1681,NaN,['Pb0.98Na0.02S'],Pb0.98Na0.02S,ThermCond,ThermCond,lattice,[823.0],823.0,Kelvin^(1.0),[823.0],[1.4],1.40,Kelvin^(-1.0) Meter^(-1.0) Watt^(1.0),NaN,NaN,({ κtot }: ∼1.4 W m−1 K−1 at 823 K for Pb0.98N...,10.1016-j.jallcom.2019.153453.xml,NaN,NaN,NaN,NaN,comma-level,κtot,An approach of enhancing thermoelectric perfor...,Elsevier,2020,no
1662,1683,NaN,['Pb0.98Na0.02S'],Pb0.98Na0.02S,ThermCond,ThermCond,total,[823.0],823.0,Kelvin^(1.0),[823.0],[0.53],0.53,Kelvin^(-1.0) Meter^(-1.0) Watt^(1.0),NaN,NaN,({ κele at 823 K }: ∼0.53 W m−1 K−1 for Pb0.98...,10.1016-j.jallcom.2019.153453.xml,NaN,NaN,NaN,NaN,comma-level,κele,An approach of enhancing thermoelectric perfor...,Elsevier,2020,no
1663,1684,NaN,['Pb0.98Na0.02S'],Pb0.98Na0.02S,ThermCond,ThermCond,lattice,[823.0],823.0,Kelvin^(1.0),[823.0],[0.9],0.90,Kelvin^(-1.0) Meter^(-1.0) Watt^(1.0),NaN,NaN,(The κlat at 823 K decreases from ∼0.9 W m−1 K...,10.1016-j.jallcom.2019.153453.xml,NaN,NaN,NaN,NaN,comma-level,κlat,An approach of enhancing thermoelectric perfor...,Elsevier,2020,no


In [506]:
tot = therm[therm.type == 'total'].copy()
ele = therm[therm.type == 'electronic'].copy()
lat = therm[therm.type == 'lattice'].copy()

In [507]:
ele.rename({'normalised_avg' : 'EleThermCond_value'}, axis=1, inplace = True)
ele.drop(columns=['normalised_model','type', 'specifier'], inplace = True)
lat.rename({'normalised_avg' : 'LatThermCond_value'}, axis=1, inplace = True)
lat.drop(columns=['normalised_model','type'], inplace = True)
tot.rename({'normalised_avg' : 'ThermCond_value'}, axis=1, inplace = True)
tot.drop(columns=['normalised_model','type'], inplace = True)

In [508]:
print((len(ele), len(lat), len(tot)))

(75, 1316, 2255)


In [509]:
ele.head()

,Unnamed: 0,merged_name,compound_name,first_name,model,normalised_temp_values,normalised_temp_avg,normalised_temp_units,temp_value,normalised_values,EleThermCond_value,normalised_units,editing,pressure,exrept,doi,error,process,direction_of_measurement,labels,parser,title,publisher,yop,oa
170,172,NaN,['PEDOT : PSS / graphene'],PEDOT : PSS / graphene,ThermCond,[295],295.0,Kelvin^(1.0),NaN,[0.3],0.30,Kelvin^(-1.0) Meter^(-1.0) Watt^(1.0),NaN,NaN,({ thermal conductivity at room temperature }:...,10.1039-c6ta00305b.html,NaN,NaN,NaN,NaN,comma-level,Use of organic solvent-assisted exfoliated MoS...,RSC,2016,no
297,300,NaN,['SrVO3−δ'],SrVO3−δ,ThermCond,[1023.0],1023.0,Kelvin^(1.0),[1023.0],[5.0],5.00,Kelvin^(-1.0) Meter^(-1.0) Watt^(1.0),NaN,NaN,(Although the thermal conductivity decreases w...,10.1016-j.jallcom.2006.02.026.xml,NaN,NaN,NaN,NaN,comma-level,Physical properties of polycrystalline SrVO3−δ\n,Elsevier,2006,no
383,387,NaN,['white petroleum jelly'],white petroleum jelly,ThermCond,"[309.0, 333.0]",321.0,Kelvin^(1.0),"[36.0, 60.0]",[91.0],91.00,Kelvin^(-1.0) Meter^(-1.0) Watt^(1.0),NaN,NaN,( (iii) a PV panel with a container of combine...,10.1016-j.enbuild.2019.109743.xml,NaN,NaN,NaN,NaN,comma-level,Can movable PCM-filled TES units be used to im...,Elsevier,2020,no
418,423,NaN,['CsAg5Te3'],CsAg5Te3,ThermCond,[727.0],727.0,Kelvin^(1.0),[727.0],[0.14],0.14,Kelvin^(-1.0) Meter^(-1.0) Watt^(1.0),NaN,NaN,(This concerted rattling phenomenon was also o...,10.1039-c9tc02029b.html,NaN,NaN,NaN,NaN,comma-level,Ultralow thermal conductivity of Tl4Ag18Te11,RSC,2019,no
441,446,NaN,['CCO single crystal'],CCO single crystal,ThermCond,[800.0],800.0,Kelvin^(1.0),[800.0],[3.0],3.00,Kelvin^(-1.0) Meter^(-1.0) Watt^(1.0),NaN,NaN,(The in-plane κ value of CCO single crystal is...,10.1016-j.mattod.2019.07.003.xml,NaN,NaN,NaN,NaN,comma-level,Layered oxygen-containing thermoelectric mater...,Elsevier,2019,no


In [510]:
dd.columns

Index(['Unnamed: 0', 'merged_name', 'compound_name', 'first_name', 'normalised_model', 'model', 'type', 'normalised_temp_values', 'normalised_temp_avg', 'normalised_temp_units', 'temp_value', 'normalised_values', 'normalised_avg', 'normalised_units', 'editing', 'pressure', 'exrept', 'doi', 'error', 'process', 'direction_of_measurement', 'labels', 'parser', 'specifier', 'title', 'publisher', 'yop', 'oa'], dtype='object')

In [511]:
#use inner merging to get only the entries which have both an electronic and lattice component
ele_lat = pd.merge(ele, lat, on=['first_name','normalised_temp_avg','doi',
                                'parser', 'oa', 'publisher', 'editing', 'pressure', 'error', 'process',
                                'labels', 'direction_of_measurement'], how='inner') # new additon keep open and pub

In [512]:
ele_lat["first_name EleThermCond_value LatThermCond_value normalised_temp_avg".split()].head()

,first_name,EleThermCond_value,LatThermCond_value,normalised_temp_avg
0,Bi1.995Cu0.005S3,0.43,0.52,573.0
1,PbS+4 % CaS+1 % PbCl2,1.20,0.40,300.0
2,Sn0.8Ge0.2Te,3.70,3.10,300.0
3,BiSbTe matrix,1.29,0.78,303.0


In [513]:
#try and find metals: resistvity between 10-6 and 10-8.
#but use normalised Conductivity
metals_check = df[(df.normalised_model == 'Conductivity') & (df.normalised_avg < (10.0**(8)))
   & (df.normalised_avg > (10.0**(6)))][ordered_columns]

metals_check.shape
# only 70 metals based on this criterion... too few :/

(71, 27)

In [514]:
def add_ele_and_lat(df):
    return (df.EleThermCond_value + df.LatThermCond_value)

In [515]:
ele_lat['SUMThermCond_value'] = ele_lat.apply(add_ele_and_lat, axis=1)

In [516]:
compare = pd.merge(tot, ele_lat, on=['first_name','normalised_temp_avg','doi'], how='inner')

In [517]:
#rename for presentability
compare.rename({'ThermCond_value':'k_tot', 'EleThermCond_value':'k_e', 'LatThermCond_value':'k_lat', 'SUMThermCond_value':'SUM'},axis=1,inplace=True)

In [518]:
#reorder
compare = compare[['first_name','k_e','k_lat','SUM','k_tot','normalised_temp_values','doi']]

In [519]:
compare  # too few, used to be more

,first_name,k_e,k_lat,SUM,k_tot,normalised_temp_values,doi
0,Sn0.8Ge0.2Te,3.7,3.1,6.8,5.4,[300.0],10.1039-c7cp04931e.html


In [520]:
dd.normalised_model.unique()

array(['ThermCond', 'ZT', 'Conductivity', 'Seebeck', 'PF'], dtype=object)

In [521]:
tot_th = therm[therm.type == 'total']

In [522]:
tot_th = tot_th.drop(['type','specifier'], axis=1)

In [523]:
tot_th.columns

Index(['Unnamed: 0', 'merged_name', 'compound_name', 'first_name', 'normalised_model', 'model', 'normalised_temp_values', 'normalised_temp_avg', 'normalised_temp_units', 'temp_value', 'normalised_values', 'normalised_avg', 'normalised_units', 'editing', 'pressure', 'exrept', 'doi', 'error', 'process', 'direction_of_measurement', 'labels', 'parser', 'title', 'publisher', 'yop', 'oa'], dtype='object')

In [524]:
#some reason tot_th.columns didn't work
cols = ['first_name',
       'normalised_avg', 'normalised_temp_values', 'parser', 'doi', 'oa', 'yop', 'publisher',
       'editing', 'pressure', 'error', 'process', 'labels', 'direction_of_measurement']  # new additon keep open and pub, and now yop

### make changes to include k_e and k_p as well

In [658]:
tot_th = therm[therm.type == 'total'][cols].rename({"normalised_avg":"k"}, axis=1).copy()
e_th = therm[therm.type == 'electronic'][cols].rename({"normalised_avg":"k_e"}, axis=1).copy()
p_th = therm[therm.type == 'lattice'][cols].rename({"normalised_avg":"k_p"}, axis=1).copy()
zt = dd[dd.normalised_model == 'ZT'][cols].rename({"normalised_avg":"ZT"}, axis=1).copy()
pf = dd[dd.normalised_model == 'PF'][cols].rename({"normalised_avg":"PF"}, axis=1).copy()
s = dd[dd.normalised_model == 'Seebeck'][cols].rename({"normalised_avg":"S"}, axis=1).copy()
el = dd[dd.normalised_model == 'Conductivity'][cols].rename({"normalised_avg":"s"}, axis=1).copy()

In [659]:
dfs = [tot_th, e_th, p_th, pf, s, el, zt]

In [660]:
for entry in dfs:
    print(entry.columns)

Index(['first_name', 'k', 'normalised_temp_values', 'parser', 'doi', 'oa', 'yop', 'publisher', 'editing', 'pressure', 'error', 'process', 'labels', 'direction_of_measurement'], dtype='object')
Index(['first_name', 'k_e', 'normalised_temp_values', 'parser', 'doi', 'oa', 'yop', 'publisher', 'editing', 'pressure', 'error', 'process', 'labels', 'direction_of_measurement'], dtype='object')
Index(['first_name', 'k_p', 'normalised_temp_values', 'parser', 'doi', 'oa', 'yop', 'publisher', 'editing', 'pressure', 'error', 'process', 'labels', 'direction_of_measurement'], dtype='object')
Index(['first_name', 'PF', 'normalised_temp_values', 'parser', 'doi', 'oa', 'yop', 'publisher', 'editing', 'pressure', 'error', 'process', 'labels', 'direction_of_measurement'], dtype='object')
Index(['first_name', 'S', 'normalised_temp_values', 'parser', 'doi', 'oa', 'yop', 'publisher', 'editing', 'pressure', 'error', 'process', 'labels', 'direction_of_measurement'], dtype='object')
Index(['first_name', 's', 'nor

In [661]:
# STATE BACKUP
df_thermcond_setup_bckp = dd.copy()

In [662]:
# RECOVER (db or dd????)
# db = df_thermcond_setup_bckp.copy()

In [663]:
# most of these combinations have a unique value, but about 200+ (for ZT) have more than one

# db.groupby(["first_name", "normalised_temp_values", "doi"]).ZT.apply(lambda S: S.dropna().unique()).reset_index().ZT.apply(len).value_counts()

In [664]:
merge_columns = [
 'first_name',
 'normalised_temp_values',
 'doi',
 'oa', 'yop',
 'parser',
 'publisher',
 'editing', 'pressure', 'process', 'labels', 'direction_of_measurement']

# added new cols which include nan values, so need to groupby with dropna = False
# added yop as well
# had error by mistake,
# but removed it, because it doesn't make sense to merge on error, it doesn't describe the compound

In [665]:
def collect(s):
    l = []
    for x in s:
        if has(x):
            l.append(x)
    return l

In [666]:
from statistics import mode, StatisticsError
try:
    mode([3,3,4,4])
except StatisticsError:
    print("Statsitcs Error")

Statsitcs Error


In [667]:
dfs_copy = dfs.copy()
dfs_collect = []

In [668]:
#for brevity. Return False for both [] and np.nan. Unfortunately bool(np.nan) evaluates to True
def has(x):
    if x == []:
        return False
    return not (x is np.nan)


In [669]:
dfs_mods = "k k_e k_p ZT PF S s".split()

# collect mutliple entries for same merge columns (e.g. same compound, same temp, diff value averages)
for dfc in dfs_copy:
    df_mod = [m for m in dfs_mods if m in dfc.columns][0]
    dfs_collect.append(dfc.groupby(merge_columns, dropna = False)[df_mod].apply(collect).reset_index())

In [670]:
#dfs_collect[0][dfs_collect[0].k.apply(lambda l: True if len(l) > 1 else False)]

In [671]:
dfs_collect[0]

,first_name,normalised_temp_values,doi,oa,yop,parser,publisher,editing,pressure,process,labels,direction_of_measurement,k
0,"(Bi,Sb)2-xSnxSe3–4xCl4x",[773.0],10.1016-j.mssp.2019.01.021.xml,no,2019,comma-level,Elsevier,NaN,NaN,NaN,NaN,NaN,[0.475]
1,(BiS)1.2(TiS2)2,[700.0],10.1016-j.rser.2018.10.006.xml,no,2019,comma-level,Elsevier,NaN,NaN,NaN,NaN,NaN,[0.3]
2,(Ce0.2Zr0.2Hf0.2Sn0.2Ti0.2)O2,[295],10.1039-c9ta05698j.html,yes,2019,comma-level,RSC,NaN,NaN,NaN,NaN,NaN,[1.28]
3,(CrSi2)0.90(WSi2)0.10,[300.0],10.1016-j.jallcom.2016.08.185.xml,no,2017,comma-level,Elsevier,WSi2 addition,NaN,sintered,NaN,NaN,[4.8]
4,(Cu0.93Ag0.07)2Se,[773.0],10.1016-j.susmat.2018.e00076.xml,no,2018,comma-level,Elsevier,NaN,NaN,NaN,NaN,NaN,[0.23]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2137,γ-Cu2S,[800.0],10.1016-j.vacuum.2019.108964.xml,no,2019,comma-level,Elsevier,NaN,NaN,high temperature,NaN,NaN,[0.19]
2138,γ-ThC2,[443.0],10.1016-B978-0-08-056033-5.00011-2.xml,no,2012,comma-level,Elsevier,NaN,NaN,NaN,NaN,NaN,[24.1]
2139,δ-Cu2S monolayer,[295],10.1039-c9nr01501a.html,no,2019,comma-level,RSC,NaN,NaN,NaN,NaN,NaN,[0.27]
2140,δ-Cu2S monolayer,[800.0],10.1016-j.vacuum.2019.108964.xml,no,2019,comma-level,Elsevier,NaN,NaN,NaN,NaN,NaN,[0.1]


In [672]:
dfs_collect[4]

,first_name,normalised_temp_values,doi,oa,yop,parser,publisher,editing,pressure,process,labels,direction_of_measurement,S
0,( 0 0 l ),[393.0],10.1016-j.cplett.2013.08.084.xml,no,2013,comma-level,Elsevier,NaN,NaN,NaN,NaN,NaN,[0.000243]
1,( 0 0 l ),"[493.0, 613.0]",10.1016-j.jallcom.2014.07.018.xml,no,2014,comma-level,Elsevier,NaN,NaN,NaN,NaN,NaN,[0.00018125]
2,( 0.01 wt % ) of Tl doped PbSe,[295],10.1016-j.mtener.2018.06.010.xml,no,2018,comma-level,Elsevier,NaN,NaN,NaN,NaN,NaN,[8.28e-05]
3,( 0.5 % Mg,[295],10.1016-j.ssc.2013.08.023.xml,no,2013,comma-level,Elsevier,NaN,NaN,NaN,NaN,NaN,[0.000102]
4,( 1 1 1 )-oriented γ-CuI films,[300.0],10.1016-j.solener.2020.10.081.xml,no,2020,comma-level,Elsevier,NaN,NaN,deposited,NaN,NaN,[0.00022]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2391,β-Zn4Sb3,[300.0],10.1039-c5ta01778e.html,no,2015,comma-level,RSC,Gd doping,NaN,NaN,NaN,NaN,[4.1e-05]
2392,β-Zn4Sb3,[300.0],10.1557-JMR.2009.0058.txt,no,2009,comma-level,Springer,NaN,NaN,NaN,NaN,NaN,[0.000117]
2393,β-Zn4Sb3,[430.0],10.1016-j.matchar.2009.01.013.xml,no,2009,comma-level,Elsevier,NaN,NaN,NaN,NaN,NaN,[0.0001275]
2394,β-Zn4Sb3,[573.0],10.1016-j.apsusc.2013.12.056.xml,no,2014,comma-level,Elsevier,NaN,NaN,NaN,NaN,NaN,[0.00036]


# Aggregate

In [673]:
from functools import reduce
import math

In [674]:
#where the magic happends: Merges the databases in pairs, on common compound, labels, doi, and temperature values
#if a value does not exist, it is populated by NaN
db = reduce(lambda df_left,df_right: pd.merge(df_left, df_right, how='outer',
                                              on=merge_columns), dfs_collect)  # new add open and publisher

# NB If there were multiple values, then they are chosen indiscriminantly, addressed this via dfs_collect

In [675]:
#db = db['compound_name first_name normalised_temp ZT S s k PF doi'.split()]

In [676]:
#get total number of NaNs
model_columns = ['k', 'k_e', 'k_p', 'PF', 'S', 's', 'ZT']
def count_nans(df):
    return df[model_columns].isnull().sum().sum()

count_nans(db)  # this has gone up since adding the k_e and k_p columns... could just get a slice and then count?

108927

In [677]:
db[['first_name', 'normalised_temp_values', 'k', 'k_e', 'k_p', 'PF', 'S', 's', 'ZT']]

,first_name,normalised_temp_values,k,k_e,k_p,PF,S,s,ZT
0,"(Bi,Sb)2-xSnxSe3–4xCl4x",[773.0],[0.475],NaN,NaN,[0.0004],NaN,NaN,[0.61]
1,(BiS)1.2(TiS2)2,[700.0],[0.3],NaN,NaN,NaN,NaN,NaN,NaN
2,(Ce0.2Zr0.2Hf0.2Sn0.2Ti0.2)O2,[295],[1.28],NaN,NaN,NaN,NaN,NaN,NaN
3,(CrSi2)0.90(WSi2)0.10,[300.0],[4.8],NaN,NaN,NaN,NaN,NaN,NaN
4,(Cu0.93Ag0.07)2Se,[773.0],[0.23],NaN,NaN,NaN,NaN,[9100.0],NaN
...,...,...,...,...,...,...,...,...,...
18504,β-Zn4Sb3 with 5 vol. % of Cu3SbSe4,[648.0],NaN,NaN,NaN,NaN,NaN,NaN,[1.37]
18505,β-Zn4Sb3-based materials,[673.0],NaN,NaN,NaN,NaN,NaN,NaN,[1.3]
18506,γ-Cu2S,[500.0],NaN,NaN,NaN,NaN,NaN,NaN,[1.2]
18507,γ-La3–δS4,[1000.0],NaN,NaN,NaN,NaN,NaN,NaN,[0.33]


In [678]:
length = 0
for dfc in dfs_collect:
    length += len(dfc)
print(length)

# sanity check two numbers must agree (the rows before aggregate, and the non nan entries in the aggregate db)
db[model_columns].size - count_nans(db)
# db[model_columns].notna().sum().sum()  # otherwise

20636


20636

In [679]:
print(np.nan == np.nan)
print(np.nan is np.nan)

False
True


In [680]:
def count_quantities(df):
    c=0
    if has(df.ZT):
        c+=1
    if has(df.S):
        c+=1
    if has(df.s):
        c+=1
    if has(df.k):  # ignoring k_e and k_p. Or should count as 1?
        c+=1
    if has(df.PF):
        c+=1
    return c

In [681]:
db['counts'] = db["ZT S s k PF".split()].apply(count_quantities, axis=1)

In [682]:
db.counts.value_counts()
#note: count is a column I made, value_counts is a standard function

1    16140
2     1187
0      931
3      210
4       38
5        3
Name: counts, dtype: int64

In [683]:
db[db.S.notna()].S.head()

14                  [0.000141]
50    [-9.999999999999999e-05]
54                   [2.9e-05]
63                    [0.0003]
66                 [0.0001787]
Name: S, dtype: object

In [684]:
#square seebeck
# db.S = db.S.apply(lambda x: (x**2.0))
db.S = db.S.apply(lambda l: [x**2.0 for x in l] if (l is not np.nan) else np.nan)

In [685]:
# make sure stuff have been squared (no negative values ofc)
db[db.S.notna()].S.head()

14    [1.9881000000000004e-08]
50     [9.999999999999999e-09]
54                  [8.41e-10]
63     [8.999999999999999e-08]
66              [3.193369e-08]
Name: S, dtype: object

In [686]:
#rename seebeck column
db = db.rename({'S': 'S_2'},axis=1)

In [687]:
# SAVE STATE
db_aggregate_backup_with_collection = db.copy()

In [688]:
db.head()

,first_name,normalised_temp_values,doi,oa,yop,parser,publisher,editing,pressure,process,labels,direction_of_measurement,k,k_e,k_p,PF,S_2,s,ZT,counts
0,"(Bi,Sb)2-xSnxSe3–4xCl4x",[773.0],10.1016-j.mssp.2019.01.021.xml,no,2019,comma-level,Elsevier,NaN,NaN,NaN,NaN,NaN,[0.475],NaN,NaN,[0.0004],NaN,NaN,[0.61],3
1,(BiS)1.2(TiS2)2,[700.0],10.1016-j.rser.2018.10.006.xml,no,2019,comma-level,Elsevier,NaN,NaN,NaN,NaN,NaN,[0.3],NaN,NaN,NaN,NaN,NaN,NaN,1
2,(Ce0.2Zr0.2Hf0.2Sn0.2Ti0.2)O2,[295],10.1039-c9ta05698j.html,yes,2019,comma-level,RSC,NaN,NaN,NaN,NaN,NaN,[1.28],NaN,NaN,NaN,NaN,NaN,NaN,1
3,(CrSi2)0.90(WSi2)0.10,[300.0],10.1016-j.jallcom.2016.08.185.xml,no,2017,comma-level,Elsevier,WSi2 addition,NaN,sintered,NaN,NaN,[4.8],NaN,NaN,NaN,NaN,NaN,NaN,1
4,(Cu0.93Ag0.07)2Se,[773.0],10.1016-j.susmat.2018.e00076.xml,no,2018,comma-level,Elsevier,NaN,NaN,NaN,NaN,NaN,[0.23],NaN,NaN,NaN,NaN,[9100.0],NaN,2


In [689]:
db.columns

Index(['first_name', 'normalised_temp_values', 'doi', 'oa', 'yop', 'parser', 'publisher', 'editing', 'pressure', 'process', 'labels', 'direction_of_measurement', 'k', 'k_e', 'k_p', 'PF', 'S_2', 's', 'ZT', 'counts'], dtype='object')

In [690]:
# these two cells no longer make sense, since we are keeping all values for each entry
# db[db.duplicated()]

In [569]:
# print(db.shape)
# db.drop_duplicates(inplace=True)
# print(db.shape)

In [691]:
db = db[['first_name', 'normalised_temp_values',
    'ZT', 'PF', 'S_2', 's', 'k', 'k_e', 'k_p',
    'counts','parser', 'doi', 'publisher', 'oa', 'yop',
    'editing', 'pressure', 'process', 'labels', 'direction_of_measurement']].copy()  # set order. Maybe should sort as well?

In [692]:
# check
for c in 'normalised_temp_values', 'ZT', 'PF', 'S_2', 's', 'k', 'k_e', 'k_p':
    print(c)
    print(db[c].apply(type).value_counts())
    print()
        

normalised_temp_values
<class 'str'>    18509
Name: normalised_temp_values, dtype: int64

ZT
<class 'list'>     10164
<class 'float'>     8345
Name: ZT, dtype: int64

PF
<class 'float'>    16507
<class 'list'>      2002
Name: PF, dtype: int64

S_2
<class 'float'>    16113
<class 'list'>      2396
Name: S_2, dtype: int64

s
<class 'float'>    15902
<class 'list'>      2607
Name: s, dtype: int64

k
<class 'float'>    16367
<class 'list'>      2142
Name: k, dtype: int64

k_e
<class 'float'>    18435
<class 'list'>        74
Name: k_e, dtype: int64

k_p
<class 'float'>    17258
<class 'list'>      1251
Name: k_p, dtype: int64



In [693]:
db.normalised_temp_values = db.normalised_temp_values.apply(make_number_list2).copy()

In [694]:
# full aggregate database, with duplicates removed 
excel_file = os.path.join(out_path, out_prefix + 'polished_aggregate_database.xlsx')
pickle_dir = os.path.join(out_path, "PICKLES")
pickle_file = os.path.join(pickle_dir, out_prefix + 'polished_aggregate_database.pkl')

if not os.path.exists(excel_file):
    db.to_excel(excel_file, index=False)
    print('created excel file')
else:
    print('excel file exists already!')
    
if not os.path.isdir(pickle_dir):
    os.mkdir(pickle_dir)
    print("(created pickle subdirectory)")
    
if not os.path.exists(pickle_file):
    db.to_pickle(pickle_file)
    print('created pickle file')
else:
    print('pickle file exists already!')

created excel file
created pickle file


In [695]:
db.head()

,first_name,normalised_temp_values,ZT,PF,S_2,s,k,k_e,k_p,counts,parser,doi,publisher,oa,yop,editing,pressure,process,labels,direction_of_measurement
0,"(Bi,Sb)2-xSnxSe3–4xCl4x",[773.0],[0.61],[0.0004],NaN,NaN,[0.475],NaN,NaN,3,comma-level,10.1016-j.mssp.2019.01.021.xml,Elsevier,no,2019,NaN,NaN,NaN,NaN,NaN
1,(BiS)1.2(TiS2)2,[700.0],NaN,NaN,NaN,NaN,[0.3],NaN,NaN,1,comma-level,10.1016-j.rser.2018.10.006.xml,Elsevier,no,2019,NaN,NaN,NaN,NaN,NaN
2,(Ce0.2Zr0.2Hf0.2Sn0.2Ti0.2)O2,[295.0],NaN,NaN,NaN,NaN,[1.28],NaN,NaN,1,comma-level,10.1039-c9ta05698j.html,RSC,yes,2019,NaN,NaN,NaN,NaN,NaN
3,(CrSi2)0.90(WSi2)0.10,[300.0],NaN,NaN,NaN,NaN,[4.8],NaN,NaN,1,comma-level,10.1016-j.jallcom.2016.08.185.xml,Elsevier,no,2017,WSi2 addition,NaN,sintered,NaN,NaN
4,(Cu0.93Ag0.07)2Se,[773.0],NaN,NaN,NaN,[9100.0],[0.23],NaN,NaN,2,comma-level,10.1016-j.susmat.2018.e00076.xml,Elsevier,no,2018,NaN,NaN,NaN,NaN,NaN


In [696]:
db.k.apply(lambda l: len(l) if l is not np.nan else 0).value_counts()

0    16367
1     2047
2       79
3       14
4        2
Name: k, dtype: int64

### Keep single values for comparison and inference. (avg)

In [697]:
# now gotta chose a way to have only unique values to make comparison. mode, avg? just skip multiples?

In [698]:
def get_mode(l):
    if isinstance(l, list):
        try:
            return mode(l)
        except StatisticsError:
            pass
    return l

In [699]:
# get averages of values (which are already averages) to perform comparison. It's simple and somewhat reasonable

# to clarify: some extractions get a range, which has been averaged
# then we got those averages, and where the merged columns agreed, added them into single entries, as part of a list
# i.e. we collected different extractions with diff avg values, for the same things, and added them into a list
# and now we get the average of that list

In [700]:
# SAVE STATE
db_aggregate_bkp = db.copy()

In [701]:
# RELOAD spot
# db = db_aggregate_bkp.copy()

In [702]:
db_avg = db.copy()

In [703]:
import statistics

In [704]:
db_avg.head(1)

,first_name,normalised_temp_values,ZT,PF,S_2,s,k,k_e,k_p,counts,parser,doi,publisher,oa,yop,editing,pressure,process,labels,direction_of_measurement
0,"(Bi,Sb)2-xSnxSe3–4xCl4x",[773.0],[0.61],[0.0004],NaN,NaN,[0.475],NaN,NaN,3,comma-level,10.1016-j.mssp.2019.01.021.xml,Elsevier,no,2019,NaN,NaN,NaN,NaN,NaN


In [705]:
mod_names = "ZT PF k S_2 s k_e k_p".split()  # do it for temp as well?

In [706]:
for m in mod_names + ["normalised_temp_values"]:
    db_avg[m] = db_avg[m].apply(lambda l: statistics.mean(l) if isinstance(l, list) else np.nan)
    

In [707]:
db_avg.normalised_temp_values.apply(type)

0        <class 'float'>
1        <class 'float'>
2        <class 'float'>
3        <class 'float'>
4        <class 'float'>
              ...       
18504    <class 'float'>
18505    <class 'float'>
18506    <class 'float'>
18507    <class 'float'>
18508    <class 'float'>
Name: normalised_temp_values, Length: 18509, dtype: object

In [708]:
db_avg[mod_names].count()

ZT     10164
PF      2002
k       2142
S_2     2396
s       2607
k_e       74
k_p     1251
dtype: int64

In [709]:
# GOTTA RENAME THE COLUMNS APPROPRIATELY?

In [710]:
# full aggregate, with flattened values using average 
excel_file = os.path.join(out_path, out_prefix + 'avg_aggregate_database.xlsx')
pickle_dir = os.path.join(out_path, "PICKLES")
pickle_file = os.path.join(pickle_dir, out_prefix + 'avg_aggregate_database.pkl')

if not os.path.exists(excel_file):
    db_avg.to_excel(excel_file, index=False)
    print('created excel file')
else:
    print('excel file exists already!')
    
if not os.path.isdir(pickle_dir):
    os.mkdir(pickle_dir)
    print("(created pickle subdirectory)")
    
if not os.path.exists(pickle_file):
    db_avg.to_pickle(pickle_file)
    print('created pickle file')
else:
    print('pickle file exists already!')

created excel file
created pickle file
